In [1]:
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import RidgeCV, SGDRegressor, LogisticRegressionCV
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import preprocessing
from tqdm.notebook import tqdm
# from autoviz.AutoViz_Class import AutoViz_Class
from pandas_profiling import ProfileReport
import datetime
from time import sleep, time
import re

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
RANDOM_SEED = 42

In [3]:
def na_info(ds: pd.DataFrame) -> pd.DataFrame:
    arr = []
    nans_cnt = ds.isna().sum()
    for ind, row in tqdm(nans_cnt.items(), total=nans_cnt.shape[0]):
        name = ind
        if ind.startswith('ind_'):
            name = buh_codes_dict[ind[4:8]]
        arr.append([ind, name, row, round(row / data.shape[0] * 100, 2)])
    return pd.DataFrame(arr, columns=['column', 'name', 'nan_cnt', 'nan_perc'])


# Корреляция между числовыми признаками набора данных
def show_num_corr(df: pd.DataFrame, features: list = None, title: str ='Корреляция между данными'):
    plt.figure(figsize=(16, 16))
    show_df =  df
    if features is not None:
        show_df = df[features]
    mask = np.triu(np.ones_like(show_df.corr(), dtype=bool))
    heatmap = sns.heatmap(show_df.corr(),
                          mask=mask, vmin=-1, vmax=1, annot=True, cmap='BrBG')
    heatmap = heatmap.set_title(title, fontdict={'fontsize': 18}, pad=16)
    return heatmap


def get_high_coreleted_features(df: pd.DataFrame, limit: float = 0.9, method: str = 'pearson') -> pd.DataFrame:
    corr_ds = df.corr(method=method)
    corr_rows = []
    for ind_row, row in corr_ds.iterrows():
        for ind_col, cell in row.items():
            if abs(cell) > limit and ind_row != ind_col:
                if [ind_col, ind_row, cell] not in corr_rows:
                    corr_rows.append([ind_row, ind_col, cell])
    return pd.DataFrame(corr_rows, columns=['feature_1', 'feature_2', 'value'])


def outliers(df: pd.DataFrame, feature: str, iqrs: float = 1.5) -> pd.DataFrame:
    # Интерквартильный размах
    perc25 = df[feature].quantile(0.25)
    perc75 = df[feature].quantile(0.75)
    IQR = perc75 - perc25
    low, high = (perc25 - iqrs * IQR, perc75 + iqrs * IQR)
    print(
        f'25-й перцентиль: {perc25}, 75-й перцентиль: {perc75}, IQR: {IQR}, Границы выбросов: [{low}, {high}]')
    return df[(df[feature] < low) | (df[feature] > high)]


def first_look(s: pd.Series) -> None:
    saved_format = pd.options.display.float_format
    pd.options.display.float_format = '{:.2f}'.format
    print('-' * 80)
    print('Название:', s.name)
    print('Тип данных:', s.dtype.type)
    print('Количество значений:', len(s))
    print('Количество уникальных значений:', s.nunique())
    print('Количество пропусков:', s.isna().sum())
    print(f'Процент пропусков: {round((s.isna().sum() / len(s)) * 100, 2)}%')
    print('Общие статистики:')
    print(s.describe())
    print('Топ 10 уникальных значений:')
    print(s.value_counts()[:10])
    print('-' * 80)
    pd.options.display.float_format = saved_format


def plot_feature_imortances(fi, columns, num=20):
    plt.rcParams['figure.figsize'] = (10, 10)
    feat_importances = pd.Series(fi, index=columns)
    mean_imp = feat_importances.nlargest(num).mean()
    feat_colors = feat_importances.apply(lambda x: 'r' if x > mean_imp else 'b')
    feat_importances.nlargest(num).plot(kind='barh', title='Feature importance')


def show_roc_auc(y_test, probs):
    fpr, tpr, threshold = roc_curve(y_test, probs)
    roc_auc = roc_auc_score(y_test, probs)
    plt.figure(figsize=(12, 8))
    plt.plot([0, 1], label='Baseline', linestyle='--')
    plt.plot(fpr, tpr, label = 'Regression')
    plt.title('Logistic Regression ROC AUC = %0.10f' % roc_auc)
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.legend(loc = 'lower right')
    plt.show()


def show_lr_Importance(model, features, n=20):
    lr_coef = pd.DataFrame(model.coef_.T, columns=['coefficient'], index=features)
    lr_coef = np.abs(lr_coef)
    lr_coef = lr_coef.sort_values(by='coefficient', ascending=False)
    f, ax = plt.subplots(1, 1, figsize=(16, 10))
    df4display = lr_coef.head(n)
    sns.barplot(x=df4display['coefficient'], y=df4display.index, palette="vlag", ax=ax)
    _ = ax.set_ylabel("Признак")
    _ = ax.set_title("Значимость признаков")


def show_confusion_matrix(y_pred, y_true):
    plt.figure(figsize=(8, 8))
    c_matrix = confusion_matrix(y_true, y_pred, labels=[1, 0])
    heatmap = sns.heatmap(c_matrix, annot=True, cmap='BrBG', fmt='g', annot_kws={"fontsize": 18})
    heatmap = heatmap.set_title("Confusion matrix", fontdict={'fontsize': 18}, pad=16)


def print_clssification_metrix(y_pred, y_true):
    print('Precision:', metrics.precision_score(y_pred, y_true))
    print('Recall:', metrics.recall_score(y_pred, y_true))
    print('F1-score:', metrics.f1_score(y_pred, y_true))


In [5]:
data = pd.read_csv('data/df_spb_all.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20310 entries, 0 to 20309
Columns: 119 entries, inn to Налог на доходы (Ф2.2410)
dtypes: float64(4), int64(109), object(6)
memory usage: 18.4+ MB


In [6]:
data.status

0        Организация ООО "СТИЛЬ" зарегистрирована в еди...
1        Организация ООО "СТИЛЬ" зарегистрирована в еди...
2        Организация ООО "СТИЛЬ" зарегистрирована в еди...
3        Организация ООО "СТИЛЬ" зарегистрирована в еди...
4        Организация ООО "СТИЛЬ" зарегистрирована в еди...
                               ...                        
20305    Организация ООО "ЗЕРКАЛО" зарегистрирована в е...
20306    Организация ООО "ЗЕРКАЛО" зарегистрирована в е...
20307    Организация ООО "ЗЕРКАЛО" зарегистрирована в е...
20308    Организация ООО "ЗЕРКАЛО" зарегистрирована в е...
20309    Организация ООО "ЗЕРКАЛО" зарегистрирована в е...
Name: status, Length: 20310, dtype: object

In [7]:
month_dict = {'января': 1, 'февраля': 2, 'марта': 3, 'апреля': 4, 'мая': 5,  'июня': 6,
              'июля': 7, 'августа': 8, 'сентября': 9,  'октября': 10,  'ноября': 11,  'декабря': 12}
def extract_reg(s: str):
    key = 'назад'
    pos = s.find(key)
    dt =  s[pos + len(key):-1].strip()
    splitted_dt = dt.split(' ')
    return datetime.date(int(splitted_dt[2]), month_dict[splitted_dt[1]], int(splitted_dt[0]))
data.status.apply(lambda x: extract_reg(x.split('\n')[0]))

0        2015-02-06
1        2015-02-06
2        2015-02-06
3        2015-02-06
4        2015-02-06
            ...    
20305    2014-11-26
20306    2014-11-26
20307    2014-11-26
20308    2014-11-26
20309    2014-11-26
Name: status, Length: 20310, dtype: object

In [8]:
data.status.apply(lambda x: np.NaN if len(x.split('\n')) < 2 else x.split('\n')[1])

0        Организация находится в стадии ликвидидации с ...
1        Организация находится в стадии ликвидидации с ...
2        Организация находится в стадии ликвидидации с ...
3        Организация находится в стадии ликвидидации с ...
4        Организация находится в стадии ликвидидации с ...
                               ...                        
20305                                                  NaN
20306                                                  NaN
20307                                                  NaN
20308                                                  NaN
20309                                                  NaN
Name: status, Length: 20310, dtype: object

In [9]:
extract_reg('Организация ООО "БСС ИНЖИНИРИНГ" зарегистрирована в едином государственном реестре юридических лиц 10 лет назад 26 мая 2011.')

datetime.date(2011, 5, 26)

In [10]:
dup_lst = ['ind_1150', 'ind_1170', 'ind_1230','ind_1300', 'ind_1310', 'ind_1320', 'ind_1350',
           'ind_1360', 'ind_1370', 'ind_1410', 'ind_1450', 'ind_1510', 'ind_1550', 'ind_2120', 'ind_2410']
for d in dup_lst:
    print([c for c in data.columns if d[4:] in c])

['ОС (Ф1.1150)']
['Фин вложения (Ф1.1170)']
['Деб задолж (Ф1.1230)']
['Итого по разделу III(Ф1.1300)']
['Уставный капитал (Ф1.1310)']
['Собственные акции, выкупленные у акционеров (Ф1.1320)']
['Добавочный капитал (Ф1.1350)']
['Резерв капитал (Ф1.1360)']
['Нераспредел прибыль (непокрытый убыток) (Ф1.1370)']
['Долгосроч заемные средства (Ф1.1410)']
['Прочие долгосрочные обязательства (Ф1.1450)']
['Заемные средства (Ф1.1510)', 'Краткосрочные заемные средства (Ф1.1510)']
['Прочие краткосрочные обязательства (Ф1.1550)']
['Себестоимость деятельности (Ф2.2120)']
['Налог на доходы (Ф2.2410)']


In [14]:
data[data['ОС (Ф1.1150)'] > 0]

,inn,ogrn,main_okved,status_org,info,status,okved,year,Отложенные налоговые активы (Ф1.1180),Итого по разделу I (Ф1.1100),...,Уставный капитал (Ф1.1310),Добавочный капитал (Ф1.1350),Резерв капитал (Ф1.1360),Нераспредел прибыль (непокрытый убыток) (Ф1.1370),Итого по разделу III(Ф1.1300),Долгосроч заемные средства (Ф1.1410),Прочие долгосрочные обязательства (Ф1.1450),Прочие краткосрочные обязательства (Ф1.1550),Себестоимость деятельности (Ф2.2120),Налог на доходы (Ф2.2410)
3,7813213777,1157847038062,62.01,"коммерческая, в процессе ликвидации","Полное наименование организации: ООО ""СТИЛЬ""\n...","Организация ООО ""СТИЛЬ"" зарегистрирована в еди...",26.20 Производство компьютеров и периферийного...,2018,0,174,...,15,0,91,282,388,0,0,0,-3240,-142
4,7813213777,1157847038062,62.01,"коммерческая, в процессе ликвидации","Полное наименование организации: ООО ""СТИЛЬ""\n...","Организация ООО ""СТИЛЬ"" зарегистрирована в еди...",26.20 Производство компьютеров и периферийного...,2019,0,54,...,15,0,373,300,688,0,0,0,0,-124
5,7813213777,1157847038062,62.01,"коммерческая, в процессе ликвидации","Полное наименование организации: ООО ""СТИЛЬ""\n...","Организация ООО ""СТИЛЬ"" зарегистрирована в еди...",26.20 Производство компьютеров и периферийного...,2020,0,85,...,15,0,673,-189,499,0,0,0,0,0
8,7838460601,1117847230082,61.10,"коммерческая, ликвидирована","Полное наименование организации: ООО ""КАПИТАЛ-...","Организация ООО ""КАПИТАЛ-СВЯЗЬ"" зарегистрирова...",43.21 Производство электромонтажных работ\n46....,2013,0,0,...,0,0,0,0,220,0,0,48,-813,-22
9,7838460601,1117847230082,61.10,"коммерческая, ликвидирована","Полное наименование организации: ООО ""КАПИТАЛ-...","Организация ООО ""КАПИТАЛ-СВЯЗЬ"" зарегистрирова...",43.21 Производство электромонтажных работ\n46....,2014,0,0,...,0,0,0,0,235,0,0,67,-977,-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20266,7842480304,1127847426805,62.01,"коммерческая, ликвидирована","Полное наименование организации: ООО ""АККАУНТО...","Организация ООО ""АККАУНТОР СОЛЮШНЗ"" зарегистри...",62.0 Разработка компьютерного программного обе...,2015,139,298,...,100,0,0,3515,3615,1992,0,488,0,-1140
20267,7842480304,1127847426805,62.01,"коммерческая, ликвидирована","Полное наименование организации: ООО ""АККАУНТО...","Организация ООО ""АККАУНТОР СОЛЮШНЗ"" зарегистри...",62.0 Разработка компьютерного программного обе...,2016,109,410,...,100,0,0,4844,4944,0,0,1279,0,-792
20270,7842480304,1127847426805,62.01,"коммерческая, ликвидирована","Полное наименование организации: ООО ""АККАУНТО...","Организация ООО ""АККАУНТОР СОЛЮШНЗ"" зарегистри...",62.0 Разработка компьютерного программного обе...,2019,294,673,...,100,0,0,9659,9759,0,0,0,-20060,-329
20272,7819320278,1147847377358,62.01,"коммерческая, действующая","Полное наименование организации: ООО ""ФИРМА ""П...","Организация ООО ""ФИРМА ""ПАССАТ"" зарегистрирова...",62.02 Деятельность консультативная и работы в ...,2014,0,2,...,15,0,2,21945,21962,0,0,0,-34180,-628


In [16]:
regex = r"^.*\(Ф\d\.(\d{4})\)"

test_str = "Финансовые и другие оборотные активы (включая дебиторскую задолженность) (Ф1.1230)"

matches = re.match(regex, test_str, re.MULTILINE)


# for matchNum, match in enumerate(matches, start=1):
#
#     print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))
#
#     for groupNum in range(0, len(match.groups())):
#         groupNum = groupNum + 1
#
#         print ("Group {groupNum} found at {start}-{end}: {group}".format(groupNum = groupNum, start = match.start(groupNum), end = match.end(groupNum), group = match.group(groupNum)))


In [17]:
regex = r"^.*\(Ф\d\.(\d{4})\)"
renames = {}
for column in data.columns:
    matches = re.match(regex, column, re.MULTILINE)
    if matches is not None:
        renames[column] = 'ind_' + matches.groups()[0]
data.rename(columns=renames, inplace=True)
renames

{'Отложенные налоговые активы (Ф1.1180)': 'ind_1180',
 'Итого по разделу I (Ф1.1100)': 'ind_1100',
 'Запасы (Ф1.1210)': 'ind_1210',
 'Налог на добавленную стоимость по приобретенным ценностям (Ф1.1220)': 'ind_1220',
 'Финансовые вложения (за исключением денежных эквивалентов) (Ф1.1240)': 'ind_1240',
 'Денежные средства и денежные эквиваленты (Ф1.1250)': 'ind_1250',
 'Прочие оборотные активы (Ф1.1260)': 'ind_1260',
 'Итого по разделу II (Ф1.1200)': 'ind_1200',
 'БАЛАНС (Ф1.1600)': 'ind_1600',
 'Итого по разделу IV (Ф1.1400)': 'ind_1400',
 'Заемные средства (Ф1.1510)': 'ind_1510',
 'Кредиторская задолженность (Ф1.1520)': 'ind_1520',
 'Оценочные обязательства (Ф1.1540)': 'ind_1540',
 'Итого по разделу V (Ф1.1500)': 'ind_1500',
 'БАЛАНС (Ф1.1700)': 'ind_1700',
 'Выручка (Ф2.2110)': 'ind_2110',
 'Валовая прибыль (убыток) (Ф2.2100)': 'ind_2100',
 'Коммерческие расходы (Ф2.2210)': 'ind_2210',
 'Управленческие расходы (Ф2.2220)': 'ind_2220',
 'Прибыль (убыток) от продаж (Ф2.2200)': 'ind_2200',

In [18]:
arr, cnt = np.unique(data.columns, return_counts=True)
arr[cnt > 1]

array(['ind_1510'], dtype=object)

In [19]:
data.columns[130:]

Index([], dtype='object')

In [20]:
for f in data.columns:
    first_look(data[f])

--------------------------------------------------------------------------------
Название: inn
Тип данных: <class 'numpy.int64'>
Количество значений: 20310
Количество уникальных значений: 3727
Количество пропусков: 0
Процент пропусков: 0.0%
Общие статистики:
count        20310.00
mean    7817145507.46
std       14241850.45
min     7801063380.00
25%     7805711680.00
50%     7813240587.00
75%     7820337678.00
max     7843318350.00
Name: inn, dtype: float64
Топ 10 уникальных значений:
7839446536    11
7813498096    11
7816504340    10
7810829462    10
7802783084    10
7811491458    10
7841441920    10
7810821022    10
7805548949    10
7814495394    10
Name: inn, dtype: int64
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Название: ogrn
Тип данных: <class 'numpy.int64'>
Количество значений: 20310
Количество уникальных значений: 3727
Количество пропусков: 0
Процент пропусков:

AttributeError: 'DataFrame' object has no attribute 'name'

In [33]:
data = pd.read_csv('data/df_spb_all.csv')
data.columns[94:106]

Index(['com_uncom', 'bankrupt', 'founding_year', 'founding_month',
       'name_okved', 'currency', 'inflation_by_2022', 'investments',
       'inflation', 'total_profit', 'breaking_even', 'payback_time'],
      dtype='object')

In [34]:
MONTH_DICT = {'января': 1, 'февраля': 2, 'марта': 3, 'апреля': 4, 'мая': 5,  'июня': 6,
              'июля': 7, 'августа': 8, 'сентября': 9,  'октября': 10,  'ноября': 11,  'декабря': 12}
def extract_reg(s: str):
    key = 'назад'
    pos = s.find(key)
    dt =  s[pos + len(key):-1].strip()
    splitted_dt = dt.split(' ')
    return datetime.date(int(splitted_dt[2]), MONTH_DICT[splitted_dt[1]], int(splitted_dt[0]))


data = pd.read_csv('data/df_spb_all.csv')

regex = r"^.*\(Ф\d\.(\d{4})\)"
renames = {}
for column in data.columns:
    matches = re.match(regex, column, re.MULTILINE)
    if matches is not None:
        renames[column] = 'ind_' + matches.groups()[0]
data.rename(columns=renames, inplace=True)

work_ds = pd.DataFrame([], index=data.index)
work_ds['inn'] = data['inn'].astype(str)
work_ds['ogrn'] = data['ogrn'].astype(str)
work_ds['main_okved'] = data['main_okved'].astype(str)
work_ds['name'] = data['info'].apply(lambda x: x.split('\n')[0][33:].strip())
work_ds['address'] = data['info'].apply(lambda x: x.split('\n')[3][18:].strip())
work_ds['type'] = data['info'].apply(lambda x: x.split('\n')[5].strip().split(',')[0].strip()[20:])
work_ds['status'] = data['info'].apply(lambda x: x.split('\n')[5].strip().split(',')[1].strip())
work_ds['okved'] = data['okved'].apply(lambda x: [str(o.split(' ')[0]) for o in x.split('\n')])
work_ds['date_reg'] = data.status.apply(lambda x: extract_reg(x.split('\n')[0]))
work_ds['comments'] = data.status.apply(lambda x: np.NaN if len(x.split('\n')) < 2 else x.split('\n')[1])
work_ds['year'] = data['year']
ind_columns = [c for c in data.columns if c.startswith('ind_')]
for ind in ind_columns:
    work_ds = pd.concat([work_ds, data[[ind]].sum(axis=1).rename(ind)], axis=1)
last_columns = data.columns[94:106]
work_ds = pd.concat([work_ds, data[last_columns]], axis=1)
work_ds.head()

work_ds.to_csv('out/df_spb_all_upd.csv', index=False)

In [35]:
data = pd.read_csv('out/df_spb_all_upd.csv')
data.head()

,inn,ogrn,main_okved,name,address,type,status,okved,date_reg,comments,...,founding_year,founding_month,name_okved,currency,inflation_by_2022,investments,inflation,total_profit,breaking_even,payback_time
0,7813213777,1157847038062,62.01,"ООО ""СТИЛЬ""","194044, г. Санкт-Петербург, пр-кт Большой Самп...",коммерческая,в процессе ликвидации,"['26.20', '26.51.8', '62.02', '62.09', '63.11'...",2015-02-06,Организация находится в стадии ликвидидации с ...,...,2015,2,Разработка компьютерного программного обеспечения,60.96,1.49,10.00,1.13,22,1,1
1,7813213777,1157847038062,62.01,"ООО ""СТИЛЬ""","194044, г. Санкт-Петербург, пр-кт Большой Самп...",коммерческая,в процессе ликвидации,"['26.20', '26.51.8', '62.02', '62.09', '63.11'...",2015-02-06,Организация находится в стадии ликвидидации с ...,...,2015,2,Разработка компьютерного программного обеспечения,67.03,1.32,11.29,1.05,57,1,1
2,7813213777,1157847038062,62.01,"ООО ""СТИЛЬ""","194044, г. Санкт-Петербург, пр-кт Большой Самп...",коммерческая,в процессе ликвидации,"['26.20', '26.51.8', '62.02', '62.09', '63.11'...",2015-02-06,Организация находится в стадии ликвидидации с ...,...,2015,2,Разработка компьютерного программного обеспечения,58.35,1.25,11.90,1.03,148,1,1
3,7813213777,1157847038062,62.01,"ООО ""СТИЛЬ""","194044, г. Санкт-Петербург, пр-кт Большой Самп...",коммерческая,в процессе ликвидации,"['26.20', '26.51.8', '62.02', '62.09', '63.11'...",2015-02-06,Организация находится в стадии ликвидидации с ...,...,2015,2,Разработка компьютерного программного обеспечения,62.71,1.22,12.20,1.04,430,1,1
4,7813213777,1157847038062,62.01,"ООО ""СТИЛЬ""","194044, г. Санкт-Петербург, пр-кт Большой Самп...",коммерческая,в процессе ликвидации,"['26.20', '26.51.8', '62.02', '62.09', '63.11'...",2015-02-06,Организация находится в стадии ликвидидации с ...,...,2015,2,Разработка компьютерного программного обеспечения,64.74,1.17,12.72,1.03,677,1,1


In [4]:
data['inn'] = data['inn'].astype(str)
data['ogrn'] = data['ogrn'].astype(str)
data['main_okved'] = data['main_okved'].astype(str)
data['name'] = data['name'].astype(str)
data['address'] = data['address'].astype(str)
data['type'] = data['type'].astype(str)
data['status'] = data['status'].astype(str)
data['okved'] = data['okved'].astype(str)
data['date_reg'] = pd.to_datetime(data['date_reg'])
data['comments'] = data['comments'].astype(str)
data['year'] = data['year'].astype(int)
data['name_okved'] = data['name_okved'].astype(str)
data.info(max_cols=200)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65374 entries, 0 to 65373
Data columns (total 146 columns):
 #    Column             Non-Null Count  Dtype         
---   ------             --------------  -----         
 0    inn                65374 non-null  object        
 1    ogrn               65374 non-null  object        
 2    main_okved         65374 non-null  object        
 3    name               65374 non-null  object        
 4    address            65374 non-null  object        
 5    type               65374 non-null  object        
 6    status             65374 non-null  object        
 7    okved              65374 non-null  object        
 8    date_reg           65374 non-null  datetime64[ns]
 9    comments           65374 non-null  object        
 10   year               65374 non-null  int64         
 11   ind_1250           65374 non-null  int64         
 12   ind_1230           65374 non-null  int64         
 13   ind_1600           65374 non-null  int64    

In [6]:
# profile = ProfileReport(data.sample(1000), title="Pandas Profiling Report", explorative=True)
# profile.to_file("out/df_moscow_all.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

KeyboardInterrupt: 

Error in callback <function flush_figures at 0x7fc0a0ca2820> (for post_execute):


KeyboardInterrupt: 

In [7]:
data.sample(10)

,inn,ogrn,main_okved,name,address,type,status,okved,date_reg,comments,...,founding_year,founding_month,name_okved,currency,inflation_by_2022,investments,inflation,total_profit,breaking_even,payback_time
26926,7733856740,1137746901852,59.11,"ООО ""СТУДИЯ 25""","125363, г. Москва, ул. Нелидовская, 9 корп. 1",коммерческая,в процессе ликвидации (принято решение о предс...,"['41.20', '43.99', '43.99.7', '59.13', '59.14'...",2013-10-02,Организация находится в стадии ликвидидации с ...,...,2013,10,"Производство кинофильмов, видеофильмов и телев...",67.0349,1.3200,22239.027822,1.0538,296,0,-1
42322,7730236717,1177746548660,62.01,"ООО ""ДИЛДЕП""","105082, г. Москва, ул. Фридриха Энгельса, 75 с...",коммерческая,действующая,"['62.02', '63.11', '63.11.1']",2017-06-02,nan,...,2017,6,Разработка компьютерного программного обеспечения,62.7078,1.2218,570.011205,1.0427,0,0,-1
32475,7708769283,1127746681820,73.1,"ООО ""ТУЙГУН ГРУПП""","107140, г. Москва, пер 1-й Красносельский, 3, ...",коммерческая,действующая,"['18.12', '59.1', '64.1', '64.9', '66.1', '68....",2012-08-30,nan,...,2012,8,Деятельность рекламная,60.9579,1.4904,499.054155,1.1291,-355,0,-1
59312,7743898002,1137746755640,62.01,"ООО ""АМЕНДА КОНСАЛТ""","125565, г. Москва, ул. Фестивальная, 4, кв. 1",коммерческая,действующая,"['62.02', '62.09', '63.11', '63.11.1', '69.20....",2013-08-21,nan,...,2013,8,Разработка компьютерного программного обеспечения,38.4217,1.6597,688.731483,1.1136,-3418,0,-1
48601,7717777837,1147746223393,62.09,"ООО ""ИНФИНИТИ""","129515, г. Москва, ул. Академика Королева, 13 ...",коммерческая,действующая,"['46.14.1', '46.15.3', '46.15.4', '46.43', '46...",2014-03-04,nan,...,2014,3,"Деятельность, связанная с использованием вычис...",67.0349,1.3200,12.573657,1.0538,94,1,3
26356,7715989716,1147746056864,62.01,"ООО ""ВАЛАЙН""","127254, г. Москва, проезд Огородный, 16 стр. 17",коммерческая,действующая,"['47.41', '47.41.4', '47.42', '47.43', '47.54'...",2014-01-29,nan,...,2014,1,Разработка компьютерного программного обеспечения,38.4217,1.6597,328.000000,1.1136,57,0,2
20167,7702381582,1157746340278,58.11,"ООО ""ИЗДАТЕЛЬСТВО ЮЛИИ ВЫСОЦКОЙ""","129110, г. Москва, пер Напрудный, 15, эт 2 пом 14",коммерческая,действующая,"['18.12', '46.19', '46.49', '46.49.3', '46.90'...",2015-04-14,nan,...,2015,4,Издание книг,67.0349,1.3200,5636.467026,1.0538,4025,0,-1
55565,7725319904,1167746549596,62.01,"ООО ""ПРОФИТ ЛАБ""","115280, г. Москва, ул. Велозаводская, 2 корп. ...",коммерческая,действующая,"['46.43', '46.49', '62.02', '62.09', '63.11', ...",2016-06-08,nan,...,2016,6,Разработка компьютерного программного обеспечения,72.1464,1.1371,10.447604,1.0491,38,1,2
26554,7717773230,1147746012061,59.11,"ООО ""АКТЯБРЬ ФИЛЬМ""","129226, г. Москва, ул. Сергея Эйзенштейна, 8 с...",коммерческая,действующая,"['18.20', '46.15.4', '47.63.1', '47.63.2', '59...",2014-01-13,nan,...,2014,1,"Производство кинофильмов, видеофильмов и телев...",62.7078,1.2218,139797.257315,1.0427,459,0,-1
40287,7728817048,1127746657455,72.1,"ООО ""КОМПОЗИТ СОЛЬЮШЕН""","117133, г. Москва, ул. Академика Варги, 8 корп...",коммерческая,действующая,"['71.20.3', '71.20.6', '71.20.9']",2012-08-24,nan,...,2012,8,Научные исследования и разработки в области ес...,38.4217,1.6597,1154.965813,1.1136,12739,1,3


In [21]:
data.sort_values(by='year').groupby('inn').first().groupby('main_okved')['investments'] #.mean()['32.12.3']

In [25]:
data_first = data.sort_values(by='year').groupby('inn').first()
data_first

,ogrn,main_okved,name,address,type,status,okved,date_reg,comments,year,...,founding_year,founding_month,name_okved,currency,inflation_by_2022,investments,inflation,total_profit,breaking_even,payback_time
inn,,,,,,,,,,,,,,,,,,,,,
1102068218,1111102000745,62.01,ООО «ИНФОЗАР»,"127055, г. Москва, пер Вадковский, 12, эт. 1 п...",коммерческая,действующая,"['46.15.4', '46.43.3', '46.90', '47.43', '47.5...",2011-04-04,nan,2011,...,2011,4,Разработка компьютерного программного обеспечения,29.3874,1.9979,68947.0,1.0610,3822,0,-1
1650329732,1161650057920,62.01,"ООО ""БОЛЬШИЕ ЧИСЛА""","115184, г. Москва, наб. Озерковская, 38-40, кв...",коммерческая,действующая,"['47.41', '47.41.4', '47.59.1', '47.63.1', '47...",2016-04-25,nan,2016,...,2016,4,Разработка компьютерного программного обеспечения,67.0349,1.3200,2074.0,1.0538,64,0,5
1655298774,1141690057134,62.01,"ООО ""РТСИМ""","121205, г. Москва, тер. Сколково Инновационног...",коммерческая,действующая,"['62.02', '72.19']",2014-08-12,nan,2014,...,2014,8,Разработка компьютерного программного обеспечения,38.4217,1.6597,6971.0,1.1136,6935,0,3
1655300448,1141690063195,62.09,"ООО ""АВАЛ,ПРАКТИКА""","115573, г. Москва, ул. Мусы Джалиля, 36 корп. ...",коммерческая,действующая,"['43.21', '46.51', '46.52', '47.41', '62.01', ...",2014-09-05,nan,2014,...,2014,9,"Деятельность, связанная с использованием вычис...",38.4217,1.6597,1241.0,1.1136,155,0,2
2310172571,1132310008797,62.09,"ООО ""УПРАВЛЯЮЩАЯ КОМПАНИЯ ХЁРСТ ШКУЛЁВ ДИДЖИТАЛ""","115114, г. Москва, ул. Дербеневская, 15Б, эт. ...",коммерческая,действующая,"['58.13', '58.14', '62.02', '63.11.1', '73.11']",2013-09-09,nan,2013,...,2013,9,"Деятельность, связанная с использованием вычис...",31.8480,1.7668,5180.0,1.0645,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9731056816,1197746690371,62.01,"ООО ""ИСИНТЕЛ""","121359, г. Москва, ул. Академика Павлова, 21 к...",коммерческая,действующая,"['62.02', '62.03', '62.09', '63.11', '63.12', ...",2019-11-25,nan,2019,...,2019,11,Разработка компьютерного программного обеспечения,64.7362,1.1718,0.0,1.0305,0,0,-1
9731057930,1197746717706,62.01,"ООО ""СИТИЛАЙФ ИННОВАЦИИ""","143026, г. Москва, тер. Сколково Инновационног...",коммерческая,действующая,"['58.29', '62.02', '62.03', '62.09', '63.11', ...",2019-12-09,nan,2019,...,2019,12,Разработка компьютерного программного обеспечения,64.7362,1.1718,0.0,1.0305,0,0,-1
9731059038,1197746743721,62.01,"ООО ""ИКК РАЗРАБОТКА МОБИЛЬНЫХ ИГР""","119361, г. Москва, ул. Марии Поливановой, 9, э...",коммерческая,действующая,"['62.02', '62.09', '63.11', '63.12']",2019-12-23,nan,2019,...,2019,12,Разработка компьютерного программного обеспечения,64.7362,1.1718,0.0,1.0305,0,0,-1


In [60]:
data_first.groupby('main_okved')['total_profit'].mean()

main_okved
20.41.4      0.000000
20.42      386.567010
23.70.2   -117.428571
26.52      296.000000
32.12      -42.239130
              ...    
91.04     -227.222222
93.1       364.571429
93.2       442.452830
94.12        0.000000
94.99        0.000000
Name: total_profit, Length: 66, dtype: float64

In [49]:
l1 = data_first.loc[data_first['main_okved'] == '61.30', 'total_profit'] # Деятельность в области спутниковой связи
l1

inn
7703411663     2834
7704798124        0
7709909783        0
7713444731      743
7714429817    12578
7716725258       39
7716896616    78503
7731385292        0
9710063516     1877
9729049530        0
Name: total_profit, dtype: int64

In [56]:

l2

inn
7709917978       0
7709993087   -2794
7714996774       0
7722444135     141
7734404070    1279
7736316895      72
7751141382     729
9710040501       0
Name: total_profit, dtype: int64

In [57]:
from scipy.stats import ttest_ind

Statistics=1.115, p=0.281


In [ ]:
data_first = data.sort_values(by='year').groupby('inn').first()
l1 = data_first.loc[data_first['main_okved'] == '61.30', 'total_profit'] # Деятельность в области спутниковой связи
l2 = data_first.loc[data_first['main_okved'] == '58.21', 'total_profit'] # Издание компьютерных игр
stat, p = ttest_ind(l1, l2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# Statistics=1.115, p=0.281